### <font color="brown">Relational Databases</font>

---

#### <font color="brown">Example 1: Nobel Prize Winners Database</font>

##### <font color="brown">We are going to build a database for the noble prize winner data that we saw previously in our discussion on JSON (Week 7).</font>

In [1]:
import json, requests

music_db_url = 'http://davidpots.com/jakeworry/017%20JSON%20Grouping,%20part%203/data.json'
resp = requests.get(music_db_url)
music_db = json.loads(resp.text)

In [2]:
print(music_db['songs'][0])

{'title': '1904', 'artist': 'The Tallest Man on Earth', 'year': '2012', 'web_url': 'http://www.songnotes.cc/songs/78-the-tallest-man-on-earth-1904', 'img_url': 'http://fireflygrove.com/songnotes/images/artists/TheTallestManOnEarth.jpg'}


---

In [1]:
import json, requests

nobel_url = 'http://api.nobelprize.org/v1/prize.json'
resp = requests.get(nobel_url)
nobels = json.loads(resp.text)

In [2]:
print(nobels['prizes'][0])

{'year': '2021', 'category': 'chemistry', 'laureates': [{'id': '1002', 'firstname': 'Benjamin', 'surname': 'List', 'motivation': '"for the development of asymmetric organocatalysis"', 'share': '2'}, {'id': '1003', 'firstname': 'David', 'surname': 'MacMillan', 'motivation': '"for the development of asymmetric organocatalysis"', 'share': '2'}]}


In [3]:
print(nobels['prizes'][0].keys())

dict_keys(['year', 'category', 'laureates'])


In [4]:
print(nobels['prizes'][1])

{'year': '2021', 'category': 'economics', 'laureates': [{'id': '1007', 'firstname': 'David', 'surname': 'Card', 'motivation': '"for his empirical contributions to labour economics"', 'share': '2'}, {'id': '1008', 'firstname': 'Joshua', 'surname': 'Angrist', 'motivation': '"for their methodological contributions to the analysis of causal relationships"', 'share': '4'}, {'id': '1009', 'firstname': 'Guido', 'surname': 'Imbens', 'motivation': '"for their methodological contributions to the analysis of causal relationships"', 'share': '4'}]}


**Each entry in the list of values for nobels['prizes'] is a dictionary of year+category+list of laureates**

In [5]:
print(nobels['prizes'][1]['laureates'][0])

{'id': '1007', 'firstname': 'David', 'surname': 'Card', 'motivation': '"for his empirical contributions to labour economics"', 'share': '2'}


##### <font color="brown">The data we want is Year, Category, First Name, Last Name, Motivation, and Share for each Nobel prize winner (laureate)</font>

##### In order to set aside most appropriate amount of space for storage in the database, we need to know maximum lengths for the motivation, first name, and last name strings. We could have done this above, but I wanted to kept it separate so it doesn't get confusing.

In [6]:
# max lengths of motivation, first name, and last name
category_maxlen=0
motiv_maxlen=0
fname_maxlen=0
lname_maxlen=0
no_laureates = []
for prize in nobels['prizes']:
    if not 'laureates' in prize:
        no_laureates.append(prize['year'])
        continue
    cat = prize['category']
    category_maxlen = max(category_maxlen, len(cat))
    for winner in prize['laureates']:
        motiv = winner.get('motivation').strip('"')
        motiv_maxlen = max(motiv_maxlen, len(motiv))
        fname = winner.get('firstname')
        fname_maxlen = max(fname_maxlen, len(fname))
        lname = winner.get('surname','')
        lname_maxlen = max(lname_maxlen, len(lname))

print(f'No laureates in the years: {no_laureates}\n')
print('Max lengths')
print(f'Category: {category_maxlen}')
print(f'Motivation: {motiv_maxlen}')
print(f'First name: {fname_maxlen}')
print(f'Last name: {lname_maxlen}')

No laureates in the years: ['1972', '1967', '1966', '1956', '1955', '1948', '1943', '1943', '1942', '1942', '1942', '1942', '1942', '1941', '1941', '1941', '1941', '1941', '1940', '1940', '1940', '1940', '1940', '1939', '1935', '1934', '1933', '1932', '1931', '1928', '1925', '1924', '1924', '1923', '1921', '1919', '1918', '1918', '1918', '1917', '1917', '1916', '1916', '1916', '1916', '1915', '1915', '1914', '1914']

Max lengths
Category: 10
Motivation: 341
First name: 59
Last name: 26


---

#### <font color="brown">Creating a new user in database system</font>

##### Execute the following commands in the MySQL client window to create the a new non-root user. You want to log in as this user when using any of the databases other than the special 'mysql' database.

- Launch terminal in Jupyter, then in the terminal log into the <tt>mysql</tt> database, which is the master database that contains all user-defined databases
    
    <pre>
    % mysql -u root -p mysql
    ...
    </pre>

- 'mysql>' is the prompt inside the MySQL session. The <tt>create user</tt> statement can then be used to create a new database system user. See https://dev.mysql.com/doc/refman/8.0/en/create-user.html <br> Give the user whatever name you like, and whatever password you want:

    <pre>
    mysql> create user 'someusername'@'localhost' identified by 'somepassword'
    ...
    </pre>

- Exit the MySQL client session

    <pre>
    mysql> exit
    </pre>

---

#### <font color="brown">Creating a nobels database</font>

##### <font color="brown">1. Create a database named nobels</font>

##### Execute the following commands in the MySQL client window to create the 'nobels' database, and grant permissions to your non-root user

- 'sesh>' is an example of a terminal prompt.<br>Log into the <tt>mysql</tt> database, which is the master database that contains all user-defined databases
    
    <pre>
    sesh> mysql -u root -p mysql
    ...
    </pre>

- The <tt>show databases</tt> statement lists all databases you have created. <br><b>Note: all commands in the mysql session end in ';'</b>

    <pre>
    mysql> show databases;
    ...
    </pre>

- Create the database with the <tt>create database</tt> statement<br>
https://dev.mysql.com/doc/refman/8.0/en/create-database.html

    <pre>
    mysql> create database nobels;
    ...
    </pre>

    <pre>
    mysql> show databases;
    ...
    </pre>
    
    You should see the nobels database in the list shown by the <tt>show databases</tt> command

- Grant 'all' (create/update/read) access to the non-root user you set up earlier (my non-root user is 'sesh')

    <pre>
    mysql> grant all on nobels.* to 'sesh'@'localhost';
    ...
    </pre>

- Exit the MySQL client session

    <pre>
    mysql> exit
    
    sesh>
    </pre>

---

##### <font color="brown">2. Create a winners table in the database</font>

##### **i) Define a couple of columns to start with - execute the following in the MySQL client window:**

- You need to first log in to the <tt>nobels</tt> database as a non-root user to whom you have granted access, using whatever password you set for that user.<br>
<b>Note: you can manipulate the nobels database when you are logged in as root to the mysql database, but it is
    not recommended. The root user has admin privileges that you might inadvertently use and thrash stuff that
    might affect other databases as well.</b>

<pre>
    sesh> mysql -u sesh -p nobels
    ...
</pre>

- Once you are logged in to the <tt>nobels</tt> database, you can see what tables there are if any (there are no tables, since we didn't make any yet), then go ahead and create a table called <tt>winners</tt><br>
    https://dev.mysql.com/doc/refman/8.0/en/create-table.html
    
<pre>
    mysql> show tables;
    Empty set (0.00 sec)

    mysql> create table winners (year year not null, category char(10) not null);
    Query OK, 0 rows affected (0.03 sec)
</pre>

- Column <tt>year</tt> has datatype <tt>year</tt><br>
https://dev.mysql.com/doc/refman/8.0/en/year.html
- Column <tt>category</tt> has datatype <tt>char(10)</tt> meaning space worth 10 characters. The actual number of characters stored in this space may be less than 10, but 10 characters of space is always set aside.<br>
https://dev.mysql.com/doc/refman/8.0/en/char.html
- When you set a column to be **not null**, it means when you add a row, that column *must* have a value, otherwise the database system will reject the add

<pre>
    mysql> show tables;
    +------------------+
    | Tables_in_nobels |
    +------------------+
    | winners          |
    +------------------+
    1 row in set (0.00 sec)
</pre>

- You can see the table schema (structure) with the <tt>desc</tt> statement:

<pre>
    mysql> desc winners;
    +----------+----------+------+-----+---------+-------+
    | Field    | Type     | Null | Key | Default | Extra |
    +----------+----------+------+-----+---------+-------+
    | year     | year(4)  | NO   |     | NULL    |       |
    | category | char(10) | NO   |     | NULL    |       |
    +----------+----------+------+-----+---------+-------+
    2 rows in set (0.00 sec)
</pre>

- Alternatively, you can use the <tt>show columns from<tt> statement<br>
   https://dev.mysql.com/doc/refman/8.0/en/show-columns.html
    
<pre>
    mysql> show columns from winners;
    +----------+----------+------+-----+---------+-------+
    | Field    | Type     | Null | Key | Default | Extra |
    +----------+----------+------+-----+---------+-------+
    | year     | year(4)  | NO   |     | NULL    |       |
    | category | char(10) | NO   |     | NULL    |       |
    +----------+----------+------+-----+---------+-------+
    2 rows in set (0.00 sec)
</pre>
    
- The default value being NULL simply means that whenever you add a row, you must supply a value. If you had chosen to NOT say *not null*, it means a null (no value) is allowed. In which case, you may specifiy a default, which would be the assumed value if none is supplied - see the **lname** column below.

##### **ii) Add in the rest of the columns**

<pre>
mysql> alter table winners add column fname varchar(80) not null;
mysql> alter table winners add column lname varchar(40);
mysql> alter table winners add column motivation varchar(500) not null;
mysql> alter table winners add column share tinyint not null;
</pre>

##### 
- The <tt>alter table</tt> statement can be used to add or modify columns <br>
https://dev.mysql.com/doc/refman/8.0/en/alter-table.html
- You don't *have* to do it this way (create with a few columns, then add in all the rest), but it is sometimes easier to break it up like this than have a super long <tt>create table</tt> statement
- The <tt>fname</tt>, <tt>lname</tt>, and <tt>motivation</tt> columns are all of type <tt>varchar</tt>, with a different number of characters, based on our earlier analysis of maximum lengths for these fields (and we added on some). Basically <tt>varchar</tt> is for variable characters, meaning the space specified is the maximum, but unlike <tt>char</tt> the actual storage will not necessarily equal the maximum specified. Instead, it will be the actual number of characters, plus a very small amount of constant extra space to tell how many actual characters there are:<br>
https://dev.mysql.com/doc/refman/8.0/en/storage-requirements.html#data-types-storage-reqs-strings
- The <tt>share</tt> column is of type </tt>tinyint</tt>, which is the least amount of integer space you can set aside for a column<br>
https://dev.mysql.com/doc/refman/8.0/en/storage-requirements.html#data-types-storage-reqs-numeric

<pre>
    mysql> show columns from winners;
    +------------+--------------+------+-----+---------+-------+
    | Field      | Type         | Null | Key | Default | Extra |
    +------------+--------------+------+-----+---------+-------+
    | year       | year(4)      | NO   |     | NULL    |       |
    | category   | char(10)     | NO   |     | NULL    |       |
    | fname      | varchar(80)  | NO   |     | NULL    |       |
    | lname      | varchar(40)  | YES  |     | NULL    |       |
    | motivation | varchar(500) | NO   |     | NULL    |       |
    | share      | tinyint(4)   | NO   |     | NULL    |       |
    +------------+--------------+------+-----+---------+-------+
    6 rows in set (0.00 sec)
</pre>

##### Key can be any column or combinaton of columns that has unique values. In this table, no column has unique values. We could do a combination of year and category as unique, since no two rows can have the same combination of year and category, but we will leave it aside for now.

---

#### <font color="brown">Loading data into winners table</font>

##### The JSON data can be added to the winners table via Python, but we will get to the Python-based database access later. For now, we will load data from a pre-done database file, nobelsv1.sql

##### Make sure the nobelsv1.sql file is in your notebook. Then, in Terminal (make sure you have exited mysql), issue the following command, to position yourself in the directory where you have this notebook. (Replace the path in this example with that on your computer. If you are Windows, your path will be something like C:\\...)
<pre>
sesh> cd /Users/seshv/courses/cs210/lectures/week12
</pre>

##### Then, issue the following command to load the data from nobelsv1.sql into the nobels database:
<pre>
sesh> mysql -u sesh -p nobels < nobelsv1.sql
</pre>

---

#### <font color="brown">Quering the winners Table</font>

##### **Queries are done using the <tt>select</tt> statement**
https://dev.mysql.com/doc/refman/8.0/en/select.html

##### **The result of any query is a table**

---

##### **1. How many entries are in the table?**

<pre>
mysql> select count(*) from winners;
+----------+
| count(*) |
+----------+
|      975 |
+----------+
</pre>

##### **2. Show the first 5 rows in the table**

<pre>
mysql> select * from winners limit 5;
+------+-----------+----------+-----------+---------------------------------------------------------------------------------+-------+
| year | category  | fname    | lname     | motivation                                                                      | share |
+------+-----------+----------+-----------+---------------------------------------------------------------------------------+-------+
| 2021 | Chemistry | Benjamin | List      | for the development of asymmetric organocatalysis                               |     2 |
| 2021 | Chemistry | David    | MacMillan | for the development of asymmetric organocatalysis                               |     2 |
| 2021 | Economics | David    | Card      | for his empirical contributions  to labour economics                            |     2 |
| 2021 | Economics | Joshua   | Angrist   | for their methodological contributions to the analysis  of causal relationships |     4 |
| 2021 | Economics | Guido    | Imbens    | for their methodological contributions to the analysis  of causal relationships |     4 |
+------+-----------+----------+-----------+---------------------------------------------------------------------------------+-------+
</pre>

**<tt>select *</tt>** selects all columns.<p>

This can be hard to read if any of the columns is extra long. In which case, an alternative is to use this variant, with a '\G' at the end of the query:
<pre>
mysql> select * from winners limit 5\G;
*************************** 1. row ***************************
      year: 2021
  category: Chemistry
     fname: Benjamin
     lname: List
motivation: for the development of asymmetric organocatalysis
     share: 2
*************************** 2. row ***************************
      year: 2021
  category: Chemistry
     fname: David
     lname: MacMillan
motivation: for the development of asymmetric organocatalysis
     share: 2
*************************** 3. row ***************************
      year: 2021
  category: Economics
     fname: David
     lname: Card
motivation: for his empirical contributions  to labour economics
     share: 2
*************************** 4. row ***************************
      year: 2021
  category: Economics
     fname: Joshua
     lname: Angrist
motivation: for their methodological contributions to the analysis  of causal relationships
     share: 4
*************************** 5. row ***************************
      year: 2021
  category: Economics
     fname: Guido
     lname: Imbens
motivation: for their methodological contributions to the analysis  of causal relationships
     share: 4
</pre>


##### **3. What are all the years for which nobel winners are listed?**

##### **4. For how many years are winners listed?**

<pre>
mysql> select count(distinct(year)) from winners;
+-----------------------+
| count(distinct(year)) |
+-----------------------+
|                   118 |
+-----------------------+
</pre>

##### **5. Who are all the winners in 2020, and for what category?**

<pre>
mysql> select category, fname, lname from winners where year=2020;
+------------+----------------------+-------------+
| category   | fname                | lname       |
+------------+----------------------+-------------+
| Chemistry  | Emmanuelle           | Charpentier |
| Chemistry  | Jennifer A.          | Doudna      |
| Economics  | Paul                 | Milgrom     |
| Economics  | Robert               | Wilson      |
| Literature | Louise               | Glück       |
| Peace      | World Food Programme |             |
| Physics    | Roger                | Penrose     |
| Physics    | Reinhard             | Genzel      |
| Physics    | Andrea               | Ghez        |
| Medicine   | Harvey               | Alter       |
| Medicine   | Michael              | Houghton    |
| Medicine   | Charles              | Rice        |
+------------+----------------------+-------------+
12 rows in set (0.00 sec)
</pre>
- Any number of columns can be specified in the **select** statement, separated by commas
- The **where** clause sets up a condition

##### **6. Who were the winners of the Literature prize in the years 2011 thru 2021?**

<pre>
mysql> select fname, lname, year from winners where category='Literature' and year between 2010 and 2020;
+------------+--------------+------+
| fname      | lname        | year |
+------------+--------------+------+
| Abdulrazak | Gurnah       | 2021 |
| Louise     | Glück        | 2020 |
| Peter      | Handke       | 2019 |
| Olga       | Tokarczuk    | 2018 |
| Kazuo      | Ishiguro     | 2017 |
| Bob        | Dylan        | 2016 |
| Svetlana   | Alexievich   | 2015 |
| Patrick    | Modiano      | 2014 |
| Alice      | Munro        | 2013 |
| Mo         | Yan          | 2012 |
| Tomas      | Tranströmer  | 2011 |
+------------+--------------+------+
</pre>
- For numeric types, you can use the **between** keyword to select range

##### **7. List all details of all the prizes that Einstein won**

<pre>
mysql> select * from winners where lname='Einstein'\G;
*************************** 1. row ***************************
      year: 1921
  category: Physics
     fname: Albert
     lname: Einstein
motivation: for his services to Theoretical Physics, and especially for his discovery of the law of the photoelectric effect
     share: 1
</pre>

---

---

#### <font color="brown">Example 2: Nobel Prize Winners Database - Version 2</font>

#### Minimizing Redundancy of Data Storage

---

##### <font color="brown">Where is the redundancy in the previous version with a single winners table?</font>

##### **1. If the nobel prize for a year is shared by multiple people, the year and category are repeated for each.**

<pre>
mysql> select count(*) from winners where share=1;
+----------+
| count(*) |
+----------+
|      353 |
+----------+

mysql> select count(*) from winners;
+----------+
| count(*) |
+----------+
|      975 |
+----------+
mysql> 
</pre>
Out of a total of 975, 353 winners are solos. So 622 winners--nearly two thirds--have shared their prize, and for each of these, there will be repeated values for year and category.

##### <font color="brown">A minimum-redundancy version would have year and category in a separate table, say **yearcat**,<br> with a *manufactured* unique integer id that would serve as a **primary key**.</font>

##### **2. There is also redundancy in the motivation, when prize is shared.**

For instance:
<pre>
select fname,lname,motivation,share from winners where year=2018 and category='Chemistry';
+----------------+--------+--------------------------------------------------+-------+
| fname          | lname  | motivation                                       | share |
+----------------+--------+--------------------------------------------------+-------+
| Frances H.     | Arnold | for the directed evolution of enzymes            |     2 |
| George P.      | Smith  | for the phage display of peptides and antibodies |     4 |
| Sir Gregory P. | Winter | for the phage display of peptides and antibodies |     4 |
+----------------+--------+--------------------------------------------------+-------+
</pre>
This is even worse than year/category redundancy since the motivation string can be fairly long. (As an aside, note that not all sharing winners are listed with the same motivation - winners could have worked in different areas for a prize in a year and category.)

##### <font color="brown">To remove this redundancy, we can make another table for motivation, say **contributon** with a *manufactured* unique integer id that would serve as the primary key.</font>

##### <font color="brown">Finally, the actual winner table, say **laureate**, would have a firstname, lastname, and share for each winner, plus a column that would be a **foreign key** that refers to the year-category in the **yearcat** table, and another column that would be another foreign key that refers to the contribution for that winner.</font>

---

#### <font color="brown">Database Schema</font>

##### 1. Log in to the **nobels** database

<pre>
sesh> mysql -u sesh -p nobels
...
</pre>
You can drop the table <tt>winners</tt> if you would like, like this (<font color="red">but don't do it yet!</font>):
<pre>
mysql> drop table winners;
</pre>
https://dev.mysql.com/doc/refman/8.0/en/drop-table.html

##### 2. Create table **yearcat**

<pre>
mysql> create table yearcat (
        id smallint auto_increment primary key, 
        year year not null, 
        category char(10) not null);
</pre>

- The datatype <tt>smallint</tt> holds a greater range of values than <tt>tinyint</tt><br>
https://dev.mysql.com/doc/refman/8.0/en/integer-types.html
- Every time you add a new row, <tt>auto_increment</tt> ensures that the <tt>id</tt> value is 1 more than the current maximum value, starting with 1 for the first row that was added.
- The <tt>id</tt> column is the primary key for this table. The database system will ensure that searches based on primary key are fast. By definition, the primary key column has unique non-null values. There can be at most one primary key for a table.

<pre>
mysql> desc yearcat;
+----------+-------------+------+-----+---------+----------------+
| Field    | Type        | Null | Key | Default | Extra          |
+----------+-------------+------+-----+---------+----------------+
| id       | smallint(6) | NO   | PRI | NULL    | auto_increment |
| year     | year(4)     | NO   |     | NULL    |                |
| category | char(10)    | NO   |     | NULL    |                |
+----------+-------------+------+-----+---------+----------------+
</pre>

##### 3. Create table **contribution**

<pre>
mysql> create table contribution (
        id smallint auto_increment primary key, 
        motivation varchar(500) not null);
        
mysql> desc contribution;
+------------+--------------+------+-----+---------+----------------+
| Field      | Type         | Null | Key | Default | Extra          |
+------------+--------------+------+-----+---------+----------------+
| id         | smallint(6)  | NO   | PRI | NULL    | auto_increment |
| motivation | varchar(500) | NO   |     | NULL    |                |
+------------+--------------+------+-----+---------+----------------+
</pre>


##### 4. Create table **laureate**

<pre>
mysql> create table laureate (
        fname varchar(80) not null, 
        lname varchar(40), 
        share tinyint not null,
        year_cat_id smallint not null,
        motiv_id smallint not null,
        foreign key (year_cat_id) references yearcat(id),
        foreign key (motiv_id) references contribution(id));
</pre>

- The columns <tt>year_cat_id</tt> and <tt>motiv_id</tt> are *dependent* on values in other tables. In this case, they are both **foriegn keys** that reference the primary key of the tables <tt>yearcat</tt> and <tt>contribution</tt>, respectively.

<pre>
mysql> desc laureate;
+-------------+--------------+------+-----+---------+-------+
| Field       | Type         | Null | Key | Default | Extra |
+-------------+--------------+------+-----+---------+-------+
| fname       | varchar(80)  | NO   |     | NULL    |       |
| lname       | varchar(40)  | YES  |     | NULL    |       |
| share       | tinyint(4)   | NO   |     | NULL    |       |
| year_cat_id | smallint(6)  | NO   | MUL | NULL    |       |
| motiv_id    | smallint(6)  | NO   | MUL | NULL    |       |
+-------------+--------------+------+-----+---------+-------+
</pre>

**Showing a table description doesn't give details of foreign key (constraint)**

In the above table description, for the keys year_cat_id and motiv_id, it says "MUL" for key, but it's not clear on what other key they depend.

**You can see the details via the <font color="brown">show create table</font> statement**
<pre>
mysql> show create table laureate\G;
*************************** 1. row ***************************
       Table: laureate
Create Table: CREATE TABLE `laureate` (
  `fname` varchar(80) NOT NULL,
  `lname` varchar(40) DEFAULT NULL,
  `share` tinyint NOT NULL,
  `year_cat_id` smallint NOT NULL,
  `motiv_id` smallint NOT NULL,
  KEY `year_cat_id` (`year_cat_id`),
  KEY `motiv_id` (`motiv_id`),
  CONSTRAINT `laureate_ibfk_1` FOREIGN KEY (`year_cat_id`) REFERENCES `yearcat` (`id`),
  CONSTRAINT `laureate_ibfk_2` FOREIGN KEY (`motiv_id`) REFERENCES `contribution` (`id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci
</pre>